**코로나19 환자의 Fake X-ray data를 사용하여 CNN 학습**

- 정상군 :1266명
- 코로나 19환자 : 1460명

# 라이브러리 및 패키지

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import time
import PIL
import tensorflow as tf

#모델 생성
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import array_to_img, img_to_array, load_img, image_dataset_from_directory
from keras.callbacks import ModelCheckpoint, EarlyStopping

#모델저장
import joblib

# 데이터셋 형성 및 이미지 전처리

In [ ]:
#구글 드라이브 mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이미지 관련 변수, 모델학습 변수

batch_size = 128
img_height = 128
img_width = 128
seed = 42
epoch = 500

In [ ]:
# Dataset 형성

train_path="/content/drive/MyDrive/CodeStates/Section4/project/After_GAN_1000/train"
test_path="/content/drive/MyDrive/CodeStates/Section4/project/After_GAN_1000/test"

train_ds = image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="training",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="validation",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds =  image_dataset_from_directory(
  test_path,
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2437 files belonging to 2 classes.
Using 1950 files for training.
Found 2437 files belonging to 2 classes.
Using 487 files for validation.
Found 433 files belonging to 2 classes.


In [ ]:
# Class 확인
class_names = train_ds.class_names
print(class_names)

['COVID19', 'NORMAL']


In [ ]:
#Train set에서 class 수 확인
num_images_train_normal = len(os.listdir(os.path.join(train_path,'NORMAL/')))
num_images_train_covid19 = len(os.listdir(os.path.join(train_path,'COVID19/')))

print(f"Normal data 수: {num_images_train_normal}")
print(f"COVID19 data 수: {num_images_train_covid19}")

Normal data 수: 1266
COVID19 data 수: 1171


In [ ]:
# 이미지 load 가볍게 해줌

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# 이미지 정규화

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# CNN 모델 형성

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(img_height, img_width, 3))) #128x128
model.add(MaxPooling2D(2,2)) 
model.add(Conv2D(64, (3,3), padding='same', activation='relu')) #64x64
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))  #32x32
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), padding='same', activation='relu')) #16x16
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 128)      0

In [ ]:
filename = 'checkpoint-epoch-{}-batch-{}-trial-001.h5'.format(epoch, batch_size)
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )

earlystopping = EarlyStopping(monitor='val_loss',  # 모니터 기준 설정 (val loss) 
                              patience=50,         # 50회 Epoch동안 개선되지 않는다면 종료
                             )

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#모델 훈련

In [ ]:
model.fit(train_ds,
          batch_size=batch_size,
          validation_data=val_ds,
          epochs=epoch,
          callbacks=[checkpoint, earlystopping])

Epoch 1/500
16/16 [==============================] - ETA: 0s - loss: 0.4675 - accuracy: 0.7923
Epoch 00001: val_loss improved from inf to 0.08911, saving model to checkpoint-epoch-500-batch-128-trial-001.h5
16/16 [==============================] - 303s 3s/step - loss: 0.4675 - accuracy: 0.7923 - val_loss: 0.0891 - val_accuracy: 0.9651
Epoch 2/500
15/16 [===========================>..] - ETA: 0s - loss: 0.0838 - accuracy: 0.9676
Epoch 00002: val_loss improved from 0.08911 to 0.02841, saving model to checkpoint-epoch-500-batch-128-trial-001.h5
16/16 [==============================] - 1s 63ms/step - loss: 0.0849 - accuracy: 0.9677 - val_loss: 0.0284 - val_accuracy: 0.9877
Epoch 3/500
15/16 [===========================>..] - ETA: 0s - loss: 0.0487 - accuracy: 0.9854
Epoch 00003: val_loss did not improve from 0.02841
16/16 [==============================] - 1s 49ms/step - loss: 0.0482 - accuracy: 0.9856 - val_loss: 0.0305 - val_accuracy: 0.9918
Epoch 4/500
15/16 [===========================

In [ ]:
model.evaluate(test_ds, verbose=2)

4/4 - 85s - loss: 0.0726 - accuracy: 0.9861 - 85s/epoch - 21s/step


[0.07264123857021332, 0.986143171787262]

#모델 저장

In [ ]:
save_path = "/content/drive/MyDrive/CodeStates/Section4/project/"
file_name = 'after_GAN_epoch_1000.pkl' 
save_path = os.path.join(save_path, file_name)

joblib.dump(model, save_path) 

INFO:tensorflow:Assets written to: ram://07d63b93-2c9f-456c-8ea3-9e2cf928d897/assets


['/content/drive/MyDrive/CodeStates/Section4/project/after_GAN_epoch_1000.pkl']